# Bootstrapping user preferences



Similarly to (Zhu et al., 2021), we'll evaluate a dynamic recommendation system using:

1. An oracle preference model (Generated in notebook 00-preference_model)
2. A position bias examination model
3. Bootstrapped user preferences

Step 3 is important so that we're simulating exposing users to fresh items, using their previous preferences (provided by step 1) and their examination bias (step 2) to determine a sinthethical rating matrix dataset that we'll use to compare different calibration approaches as the user gets exposed to more and more items.

We'll also simulate the timestamp of user interaction, with the following methodology:

1. We'll analyze the average delta in timestamps between consecutive ratings per user in the same dataset that the preference model was trained on (e.g: movielens) (present in data/movielens-1m/avg_str_time_dff_per_user.csv)
2. Assuming that the time_delta is normally distributed (with the mean and standart deviation being defined from the dataset), we'll randomly generate a delta_t for each user. the time_0 will be the timestamp when we start generating the user preferences.

In [1]:
import pandas as pd
from scipy.stats import expon
from tqdm import tqdm

In [2]:
import numpy as np
import math
import random

In [3]:
import sys
import os
from pathlib import Path
import pickle

sys.path.append('..')

In [4]:
from simulationConstants import ML_1M_FILLED_PATH

## Reading and setting up data

In [5]:
preference_matrix = pd.read_csv(f"../{ML_1M_FILLED_PATH}")
preference_matrix = preference_matrix.rename(columns={"Rating": "rating"})


unique_users = list(preference_matrix["user"].unique())

unique_items = list(preference_matrix["item"].unique())


click_matrix = pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at", "timestamp"])

user_to_time_delta = pd.read_csv("../data/movielens-1m/median_time_diff_per_user.csv").set_index("userId")

In [6]:


userToExpDistribution = {
    user: expon(scale=row["median_timestamp_diff"])
    for user, row in user_to_time_delta.iterrows()
}




with open("userToExpDistribution.pkl", "wb") as f:
    pickle.dump(userToExpDistribution, f)

## Functions

In [ ]:
from simulationUtils import bootstrap_clicks

In [9]:
initial_date = None

In [10]:
if initial_date is None:
    initial_date = pd.Timestamp.now().timestamp()
user_to_up_to_date_timestamp = pd.DataFrame({
    "user": unique_users, 
    "delta_from_start": 0.0
})
user_to_up_to_date_timestamp["timestamp_dist"] = user_to_up_to_date_timestamp["user"].map(userToExpDistribution)

In [ ]:
%%time
bootstrapped_clicks = bootstrap_clicks(click_matrix, unique_users, unique_items, preference_matrix, userToExpDistribution=userToExpDistribution, initial_date=0.0, rounds=10)

Processing users (round 1/10)...: 100%|██████████| 5289/5289 [14:08<00:00,  6.23it/s]
/tmp/ipykernel_16597/2071917566.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, round_df], ignore_index=True)
Processing users (round 9/10)...:   8%|▊         | 438/5289 [01:12<14:21,  5.63it/s]

In [ ]:
bootstrapped_clicks

,user,item,clicked_and_examined,clicked_at,timestamp


In [ ]:
bootstrapped_clicks.to_csv("../data/simulation/movielens_1m_sinthetically_filled.csv", index=False)